In [26]:
import requests
import json
import pprint

In [27]:
pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [331]:
from elasticsearch import Elasticsearch
from io import StringIO
import pandas as pd

In [397]:
es = Elasticsearch('http://elasticsearch:9200')

(?)!!!!!Mirar cómo poner size infinito!!!!!!!(?)

In [411]:
res = es.search(index="logs-training", body={"query": {"match_all": {}}}, size=100)

In [412]:
df_empty = pd.DataFrame(columns = ["index","method","url","protocol","userAgent","pragma","cacheControl","accept","acceptEncoding","acceptCharset","acceptLanguage","host","connection","contentLength","contentType","cookie","payload","label"])



In [413]:
for i in range(int(len(res["hits"]["hits"]))):
    message = res["hits"]["hits"][i]["_source"]["message"]
    data = StringIO(message)
    df = pd.read_csv(data, sep = ",", names = ["index","method","url","protocol","userAgent","pragma","cacheControl","accept","acceptEncoding","acceptCharset","acceptLanguage","host","connection","contentLength","contentType","cookie","payload","label"])
    df_empty = df_empty.append(df, ignore_index=True)

In [414]:
df_empty

,index,method,url,protocol,userAgent,pragma,cacheControl,accept,acceptEncoding,acceptCharset,acceptLanguage,host,connection,contentLength,contentType,cookie,payload,label
0,5021,GET,http://localhost:8080/tienda1/publico/registro...,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=42C5F69573A6487AF877BD05DED50143,direccion=Segura+30%2C+,norm
1,5021,GET,http://localhost:8080/tienda1/publico/registro...,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=42C5F69573A6487AF877BD05DED50143,ciudad=Bidaurreta,norm
2,5021,GET,http://localhost:8080/tienda1/publico/registro...,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=42C5F69573A6487AF877BD05DED50143,cp=34829,norm
3,5021,GET,http://localhost:8080/tienda1/publico/registro...,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=42C5F69573A6487AF877BD05DED50143,provincia=Guip%FAzcoa,norm
4,5021,GET,http://localhost:8080/tienda1/publico/registro...,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=42C5F69573A6487AF877BD05DED50143,ntc=7332706873817194,norm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5075,GET,http://localhost:8080/tienda1/publico/pagar.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=4BA16A9603EE4785FF3BD17703C7E5FE,modo=insertar,norm
96,5075,GET,http://localhost:8080/tienda1/publico/pagar.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=4BA16A9603EE4785FF3BD17703C7E5FE,precio=7785,norm
97,5075,GET,http://localhost:8080/tienda1/publico/pagar.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=4BA16A9603EE4785FF3BD17703C7E5FE,B1=Confirmar,norm
98,5076,GET,http://localhost:8080/tienda1/publico/producto...,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=FA4D98F4D272C07CA804A64196C9BEA7,NaN,norm
